In [ ]:
#Calculating Phrase Similarity

In [1]:
import gensim
from gensim.models import KeyedVectors

# Load Word2Vec vectors
location = "C:/Users/reena/Downloads/Re/GoogleNews-vectors-negative300.bin.gz"
wv = KeyedVectors.load_word2vec_format(location, binary=True, limit=1000000)
wv.save_word2vec_format('vectors.csv')

<h1>Calculating Phrase Similarity

In [5]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors

# Load Word2Vec vectors
location = "C:/Users/reena/Downloads/Re/GoogleNews-vectors-negative300.bin.gz"
wv = KeyedVectors.load_word2vec_format(location, binary=True, limit=1000000)
wv.save_word2vec_format('vectors.csv')

# Load phrases from CSV with specified encoding
phrases_df = pd.read_csv("C:/Users/reena/Downloads/Re/phrases (1).csv", encoding='latin1')

# Function to calculate phrase embeddings
def get_phrase_embedding(phrase, model):
    tokens = word_tokenize(phrase.lower())
    tokens = [token for token in tokens if token in model]
    if not tokens:
        return None
    phrase_embedding = sum(model[token] for token in tokens) / len(tokens)
    return phrase_embedding

# Calculate similarity between phrases
similarities = []
for i, phrase1 in enumerate(phrases_df['Phrases']):
    phrase1_embedding = get_phrase_embedding(phrase1, wv)
    if phrase1_embedding is None:
        continue
    for j, phrase2 in enumerate(phrases_df['Phrases']):
        phrase2_embedding = get_phrase_embedding(phrase2, wv)
        if phrase2_embedding is None:
            continue
        cosine_sim = cosine_similarity([phrase1_embedding], [phrase2_embedding])[0][0]
        similarities.append((i, j, cosine_sim))
# reena, try to make it shorter and use GPtto look for ways to amke it shorter 


# Store results
similarity_df = pd.DataFrame(similarities, columns=['Phrase_1', 'Phrase_2', 'Similarity'])
similarity_df.to_csv('phrase_similarities.csv', index=False)

# Function to find closest match for user input
def find_closest_match(user_input, phrases_df, model):
    input_embedding = get_phrase_embedding(user_input, model)
    if input_embedding is None:
        return "Input not found in embeddings"
    
    similarities = []
    for i, phrase in enumerate(phrases_df['Phrases']):
        phrase_embedding = get_phrase_embedding(phrase, model)
        if phrase_embedding is None:
            continue
        cosine_sim = cosine_similarity([input_embedding], [phrase_embedding])[0][0]
        similarities.append((i, cosine_sim))
    
    similarities.sort(key=lambda x: x[1], reverse=True)
    closest_match_index, similarity_score = similarities[0]
    closest_match_phrase = phrases_df.iloc[closest_match_index]['Phrases']
    
    return closest_match_phrase, similarity_score


<H1>Bouns

### Commentary and Future Improvements:

1. **Code Structuring**: The above wrtten code can be organized into separate modules for data loading, processing, similarity calculation, and app functionality. This separation improves readability, maintenance, and testing.

2. **Optimization**: To improve performance, consider optimizing the code for memory efficiency, possibly by using more efficient data structures or algorithms. Also, explore vectorization and parallelization for faster computation, especially with large datasets.

3. **Error Handling**: Enhance error handling by incorporating more specific exceptions, logging, and informative error messages. This helps in debugging and maintaining code in the future.

4. **Unit Testing**: Expand unit tests to cover different aspects of the code, especially critical functions and edge cases. Use frameworks like `pytest` to automate testing and ensure code reliability.

5. **Parameterize Configurations**: Use configuration files (JSON, YAML, etc.) to parameterize settings like file paths, model configurations, or hyperparameters. This makes the code more flexible and easier to maintain.

6. **Containerization with Docker**: Dockerize the application and its dependencies for easy deployment and reproducibility across different environments. Include a Dockerfile and, if necessary, a `docker-compose.yml` for multiple services (like app and database)

In [ ]:
# sample code forr handling error in above case - 
import logging

class CustomDataError(Exception):
    """Custom exception for data-related errors."""
    pass

def data_processing_function(data):
    try:
        # Processing code...
        if not data:
            raise CustomDataError("Empty data received for processing")
        # More data processing...
    except CustomDataError as e:
        # Logging the error with specific details
        logging.error(f"Data processing error: {e}")
        raise  # Re-raise the custom exception for higher-level handling
    except Exception as e:
        # Generic exception handling and logging
        logging.exception(f"Unexpected error: {e}")
        raise  # Re-raise the exception for higher-level handling


### Optimizing Data Processing:

Chunking the data and processing it in parallel can significantly improve efficiency, especially with large datasets. Libraries like `dask` or `joblib` can assist in parallelizing tasks across multiple CPU cores or nodes. Splitting data into smaller chunks, processing them concurrently, and aggregating the results can reduce computation time.


### Structuring Code as a Pipeline:

1. **Orchestration vs. Manipulation**: Define separate classes or functions for orchestration (pipeline control) and data manipulation. This separation adheres to a more modular design where each step in the pipeline is distinct and easily replaceable.

2. **Data Manipulation & I/O**: Have modules dedicated to data manipulation and I/O operations, allowing easy modification or exchange of these components without affecting other parts of the pipeline.

3. **Configuration**: Centralize configurations for I/O (file paths, database connections) and pipeline execution parameters. This flexibility enables easy modification without code changes.


### Future Ideas for Phrase Processing:

1. **Clean Data**: Remove duplicates, outliers, and stopwords from phrases for more accurate similarity calculations.

2. **Out-of-Vocabulary Handling**: Implement strategies for handling out-of-vocabulary words in Word2Vec. Techniques like Levenshtein distance can help find similar words and use their vectors if an exact match isn't found.